In [ ]:
%%time
import pandas as pd
from file_paths_and_consts import *

SHAPEFILE_LOC = 'ukr_shp/ukr_admbnda_adm2_sspe_20230201.shp'
ID_COLUMN = 'ADM2_EN'
HOUSEHOLD_LOC = 'ukr_household.csv'
PERSON_LOC = 'ukr_person.csv'
HOUSEHOLD_PERSON_MATCHING = 'ukr_residence_location_assignment.csv'
HH_FILE_PREFIX = 'ukraine_household_data_ADM2_HDX.csv'



In [ ]:
%%time
import geopandas as gpd
import warnings
warnings.filterwarnings('ignore')

shape_gdf = gpd.read_file(SHAPEFILE_LOC)
shape_gdf = shape_gdf.rename(columns={ID_COLUMN:'ADM2_EN'})

shape_gdf_trimmed = shape_gdf[['ADM2_EN','geometry']]

In [ ]:
%%time

def demographic_type(age,sex): ##this function may be played with
    if(age>old_age):
        return 0
    if(age<child_age):
        return 1
    if(sex==1):
        return 2
    return 3

hh_df = pd.read_csv(UNCLEANED_DATA_DIR+HOUSEHOLD_LOC)
person_df = pd.read_csv(UNCLEANED_DATA_DIR+PERSON_LOC)
ukr_house_loc = pd.read_csv(UNCLEANED_DATA_DIR+HOUSEHOLD_PERSON_MATCHING)

ukr_house_loc_gdf = gpd.GeoDataFrame(ukr_house_loc[['hid','rlid','latitude','longitude','admin1','admin2']], geometry=gpd.points_from_xy(ukr_house_loc.longitude, ukr_house_loc.latitude))
hh_matched_admin2_raion = gpd.sjoin(ukr_house_loc_gdf,shape_gdf,op='within')

print(hh_df.shape,person_df.shape)

hh_df = hh_df.dropna(thresh = hh_df.shape[0]/2, axis = 1)
person_df = person_df.dropna(thresh = person_df.shape[0]/2, axis = 1)

print(hh_df.shape,person_df.shape)
print(hh_matched_admin2_raion.shape)


In [ ]:
%%time
old_age = 60
child_age = 18

person_df['type'] = person_df.apply(lambda x: demographic_type(x['age'],x['sex']),axis=1)

for i in range(0,len(DEMO_TYPES)):
    person_df[DEMO_TYPES[i]] = person_df['type'].apply(lambda x: 1 if x == i else 0)
print('for loop done')
## COUNT FOR EACH HOUSE EACH DEMO TYPE
hid_with_demo_count_df = person_df.groupby('hid')[DEMO_TYPES].sum().reset_index()

hh_with_actual_raion_matching = hid_with_demo_count_df[hid_with_demo_count_df.hid.isin(hh_matched_admin2_raion.hid.tolist())]
hh_with_actual_raion_matching = hh_with_actual_raion_matching.sort_values(by='hid')
hh_matched_admin2_raion = hh_matched_admin2_raion.sort_values(by='hid')
hh_with_actual_raion_matching_final = hh_with_actual_raion_matching.merge(hh_matched_admin2_raion,on='hid',how='inner')
print('merge done')
hh_with_actual_raion_matching_final['matching_place_id'] = hh_with_actual_raion_matching_final['ADM2_EN']
hh_with_actual_raion_matching_final.to_csv(HOUSEHOLD_DIR+HH_FILE_PREFIX,index=False)